In [68]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [69]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")

from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [70]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
    ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
    translation = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/openie_result.pickle', mode="rb") as f:
    openie = pickle.load(f)

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
BASE = Namespace("http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/")
#EMAIL = Namespace("http://www.w3.org/2000/10/swap/pim/email#")
OLIA = Namespace("http://purl.org/olia/olia.owl#")
NERD = Namespace("http://nerd.eurecom.fr/ontology#")
NIF = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
ITSRDF = Namespace("https://www.w3.org/2005/11/its/rdf#")
WD = Namespace("http://www.wikidata.org/entity/")

In [ ]:
# RDFグラフ
g = Graph()
g.bind('',BASE)
g.bind('schema',SDO)
g.bind('foaf',FOAF)
g.bind('owl',OWL)
g.bind('rdf',RDF)
g.bind('rdfs',RDFS)
g.bind('olia',OLIA)
g.bind('nerd',NERD)
g.bind('nif',NIF)
g.bind('itsrdf',ITSRDF)
g.bind('wd',WD)

In [89]:
oie_triples = []
# メールごとにopenIEの結果を取り出す
for uid, oie_results in openie.items():
    # openIEの結果とエンティティの抽出結果があるときのみ,処理を継続する
    if oie_results == [] or tagme_result[uid] is None:
        pass
    else:
        # メールの文ごとにopenIEの結果を取り出す
        for result in oie_results:
            sentence = result['sentence']
            sent_start = result['start']
            facts = result['facts']
            # トリプルを取り出す
            for fact in facts:
                s = fact['subject']
                p = fact['predicate']
                o = fact['object']
                # subjectとobjectのオフセットを取得する
                s_match = re.search(re.escape(s),sentence)
                o_match = re.search(re.escape(o),sentence)
                if s_match is None:
                    pass
                else:
                    s_start = s_match.start() + sent_start
                    s_end = s_match.end() + sent_start
                    # tagmeのアノテーション結果を走査する
                    for ann in tagme_result[uid]['annotations']:         
                        # トリプルの主語とspot, start, endが一致する場合，トリプル（主語，メンション，検出されたスポットのリソース）作成
                        if s == ann['spot'] and s_start == ann['start'] and s_end == ann['end'] and ann['link_probability'] >= 0.3:
                            #URI参照される表記に変換する
                            emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid)
                            s = ':' + re.sub(r'[^a-zA-Z_0-9]','_',s)
                            offset_based_string = emailmessage + '#offset_' + str(s_start) + '_' + str(s_end)
                            oie_triples.append((s,'mentions',offset_based_string))
                            if o_match is None:
                                if re.search(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',o) or re.search(r'[\w\-._]+@[\w\-._]+\.[A-Za-z]+',o):
                                    p = ':' + re.sub(r'[^a-zA-Z_0-9]','_',p)
                                    oie_triples.append((s,p,o))
                                else:
                                    o = re.sub(r'[^a-zA-Z_0-9]','_',o) + '@en'
                                    p = ':' + re.sub(r'[^a-zA-Z_0-9]','_',p)
                                    oie_triples.append((s,p,o))
                            else:
                                o_start = o_match.start() + sent_start
                                o_end = o_match.end() + sent_start
                                resource_check = False
                                # tagmeのアノテーション結果を走査する
                                for ann in tagme_result[uid]['annotations']:
                                    #　トリプルの目的語と一致するspotがある場合，（目的語，メンション，検出されたスポットのリソース）作成
                                    if o == ann['spot'] and o_start == ann['start'] and o_end == ann['end'] and ann['link_probability'] >= 0.3:
                                        o = ':' + re.sub(r'[^a-zA-Z_0-9]','_',o)
                                        offset_based_string = emailmessage + '#offset_' + str(o_start) + '_' + str(o_end)
                                        oie_triples.append((o,'mentions',offset_based_string))
                                        p = ':' + re.sub(r'[^a-zA-Z_0-9]','_',p)
                                        oie_triples.append((s,p,o))
                                        resource_check = True
                                #openIEのトリプル作成
                                if resource_check == True:
                                    continue
                                else:
                                    if re.search(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',o) or re.search(r'[\w\-._]+@[\w\-._]+\.[A-Za-z]+',o):
                                        p = ':' + re.sub(r'[^a-zA-Z_0-9]','_',p)
                                        oie_triples.append((s,p,o))
                                    else:
                                        o = re.sub(r'[^a-zA-Z_0-9]','_',o) + '@en'
                                        p = ':' + re.sub(r'[^a-zA-Z_0-9]','_',p)
                                        oie_triples.append((s,p,o))

In [90]:
oie_triples

[(':Google', 'mentions', '4084867E_9050502_kbc_net_au#offset_400_406'),
 (':Google', ':is', 'search_engine@en'),
 (':Google', 'mentions', '4084867E_9050502_kbc_net_au#offset_538_544'),
 (':Google', ':has', 'great_product@en'),
 (':Google', 'mentions', '4084867E_9050502_kbc_net_au#offset_538_544'),
 (':Google', ':has', 'great_product@en'),
 (':Google', 'mentions', '4084867E_9050502_kbc_net_au#offset_717_723'),
 (':Google', ':fail_as', 'company@en'),
 (':Google', 'mentions', '4084867E_9050502_kbc_net_au#offset_717_723'),
 (':Google', ':start', 'charging_excessively@en'),
 (':html',
  'mentions',
  '406ff2ad_827024438_smtp_bjoern_hoehrmann_de#offset_628_632'),
 (':html', ':is', 'something@en'),
 (':spam',
  'mentions',
  '20030612133301673526_GyazMail_josh_deaghean_com#offset_434_438'),
 (':spam', ':be', 'external@en'),
 (':Google',
  'mentions',
  '200102101407_f1AE79o20047_djwhome_demon_co_uk#offset_1_7'),
 (':Google', ':provides_plain_text_version_of', 'document@en'),
 (':Word_Windows'